In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import scipy as sc
import cupy as cp
import numba as nb
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import Reshape, MaxPooling2D
from tensorflow.python.keras.layers import AveragePooling2D, BatchNormalization
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.callbacks import Callback
from tensorflow.python.keras.optimizers import Adam


from tensorflow.python.keras.layers import LeakyReLU, Dropout
from numba import autojit
from sklearn.metrics import roc_auc_score, auc, roc_curve
from numpy.linalg import multi_dot
import warnings
warnings.filterwarnings("ignore")


In [9]:
bkgnd = 0;
J = 64*64;
Jsq = np.sqrt(J);
Jsq = Jsq.astype(int);
gbar1 = bkgnd*np.ones((J,1));
gbar2 = gbar1;

@autojit
def MakeCovar(sigma,Jsq):
  coords = np.square(np.linspace(-(Jsq/2),((Jsq/2)),(Jsq),endpoint = False))
  origin = np.nonzero(coords==0);
  origin = origin[0]
  dsq = sc.linalg.circulant(coords);
  dsq[0:(origin[0]),:]=np.flip(dsq[0:(origin[0]),:])
  dsq[(origin[0]):(Jsq),:]=np.flip(dsq[(origin[0]):(Jsq),:])
  K = np.exp(-(1/(2*(sigma**2)))*dsq);
  K = np.kron(K,K);
  return K

In [10]:
K = 500; # Number of Images
N_test = 10000;
sigma1 = .38;
sigma2 = .3;
K1 = MakeCovar(sigma1,Jsq);
K2 = MakeCovar(sigma2,Jsq);
K1half = np.real(sc.linalg.sqrtm(K1));
K2half = np.real(sc.linalg.sqrtm(K2));
g1test = np.zeros((N_test,J)); #Flattened image array. [K,J]
g2test = np.zeros((N_test,J)); #Flattened image array. [K,J] ex. [1000,2500]
g1samp  = (np.dot(K1half,np.random.normal(loc = 0, scale = 1, size = (J,K)))).T;
g2samp = (np.dot(K2half,np.random.normal(loc = 0, scale = 1, size = (J,K)))).T;
g1test = (np.dot(K1half,np.random.normal(loc = 0, scale = 1, size = (J,N_test)))).T;
g2test = (np.dot(K2half,np.random.normal(loc = 0, scale = 1, size = (J,N_test)))).T;


In [14]:
#Creating labels and randomize them:
g_total_train = np.zeros([np.int(K),J])
g_label_train = np.zeros([np.int(K),1])

g_total_test = np.zeros([np.int(N_test),J])
g_label_test = np.zeros([np.int(N_test),1])

g_total_train[0:np.int(K/2),:] = g1samp[0:np.int(K/2),:]
g_total_train[np.int(K/2):np.int(K),:] = g2samp[0:np.int(K/2),:]

g_total_test[0:np.int(N_test/2),:] = g1test[np.int(N_test/2):np.int(N_test),:]
g_total_test[np.int(N_test/2):np.int(N_test),:] = g2test[np.int(N_test/2):np.int(N_test),:]
g_label_train[0:np.int(K/2)] = 1;
g_label_train[np.int(K/2):K] = 0;

g_label_test[0:np.int(N_test/2)] = 1;
g_label_test[np.int(N_test/2):K] = 0;


def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
[g_total_train,g_label_train] = unison_shuffled_copies(g_total_train,g_label_train)
[g_total_test,g_label_test] = unison_shuffled_copies(g_total_test,g_label_test)

In [15]:
#CNN Info
# The number of pixels in each dimension of an image.
img_size = Jsq;

# The images are stored in one-dimensional arrays of this length.
img_size_flat = J;

# Tuple with height and width of images used to reshape arrays.
img_shape = (Jsq,Jsq)

# Tuple with height, width and depth used to reshape arrays.
# This is used for reshaping in Keras.
img_shape_full = (Jsq,Jsq,1)

# Number of classes, one class for each of 10 digits.
num_classes = 2

# Number of colour channels for the images: 1 channel for gray-scale.
num_channels = 1

In [19]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
encoder = LabelEncoder()
encoder.fit(np.ravel(g_label_train))
encoded_Y = encoder.transform(np.ravel(g_label_train))
encoder2 = LabelEncoder()
encoder.fit(np.ravel(g_label_test))
encoded_Y_test = encoder.transform(np.ravel(g_label_test))
with tf.device('/device:GPU:2'):
    model = Sequential()

    model.add(InputLayer(input_shape=(img_size_flat)))

    model.add(Reshape(img_shape_full))

    #First convolutional layer with ReLU-activation and max-pooling.
    model.add(Conv2D(kernel_size=2, strides=1, filters=32, activation = 'relu',padding='same'
                    , name='layer_conv1'))

    # Flatten the 4-rank output of the convolutional layers
    # to 2-rank that can be input to a fully-connected / dense layer.
    model.add(Flatten())
    model.add(Dense((2^8),activation = 'relu'))

    # Last fully-connected / dense layer with softmax-activation
    # for use in classification.
    model.add(Dense(1, activation='softmax'))

    optimizer = Adam(lr=1e-3)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                    metrics=['acc'])

    
    model.fit(x=g_total_train,y=encoded_Y,validation_data=(g_total_test, encoded_Y_test),
               shuffle = True,
              epochs=1000, batch_size=1000)
    y_pred_keras = model.predict(g_total_test).ravel()
    fpr_keras, tpr_keras, thresholds_keras = roc_curve(g_label_test, y_pred_keras)
    auc_keras = auc(fpr_keras, tpr_keras)
    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(loc='best')

ValueError: Tried to convert 'y' to a tensor and failed. Error: None values not supported.

In [ ]:
y_pred_keras = model.predict(g_total_test).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(g_label_test, y_pred_keras)
auc_keras = auc(fpr_keras, tpr_keras)
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')